In [18]:
# for reloading the editable module
%load_ext autoreload
%autoreload 2

# Register your package and submodules
# %aimport probabilistic_load_forecast
# %aimport probabilistic_load_forecast.application.services
# %aimport probabilistic_load_forecast.adapters.db
# %aimport probabilistic_load_forecast.adapters.cds
# %aimport probabilistic_load_forecast.adapters.entsoe
# %aimport probabilistic_load_forecast.domain.model
from pathlib import Path
from dotenv import load_dotenv
from datetime import timezone, datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import stats
# Import services and domain models
from probabilistic_load_forecast.application.services import (
    GetActualLoadData,
    GetERA5DataFromDB,
    GetERA5DataFromCDSStore,
    CreateCDSCountryAverages,
)

from probabilistic_load_forecast.domain.model import (
    LoadMeasurement,
    LoadTimeseries,
    Era5Timeseries,
)

# Import adapters for data access
from probabilistic_load_forecast.adapters.db import (
    EntsoePostgreRepository,
    Era5PostgreRepository,
)

from probabilistic_load_forecast.adapters.entsoe import (
    EntsoeAPIClient,
    EntsoeFetcher,
    XmlLoadMapper,
    EntsoeDataProvider,
)

from probabilistic_load_forecast.adapters.cds import (
    CDSAPIClient,
    CDSConfig,
    CDSDataProvider,
    FileRepository
)

# Import configuration
from probabilistic_load_forecast import config

# Load environment variables

if not load_dotenv(Path("../..") / ".env"):
    raise FileNotFoundError("Could not open the .env file.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# Initialize database repositories
try:
    # Load data repository
    load_repo = EntsoePostgreRepository(config.get_postgre_uri())

    # ERA5 weather data repository
    era5_repo = Era5PostgreRepository(config.get_postgre_uri())

    print("Database repositories initialized successfully")
except Exception as e:
    print(f"Error initializing repositories: {e}")
    print("Make sure your .env file contains the correct database credentials")

Database repositories initialized successfully


In [69]:
# Initialize services
get_load_data = GetActualLoadData(load_repo)
get_era5_data = GetERA5DataFromDB(era5_repo)

print("Services initialized successfully")

Services initialized successfully


In [70]:
start = datetime(2018, 10, 1, 0, 0, tzinfo=timezone.utc)
end = datetime(2025, 10, 10, 0, 0, tzinfo=timezone.utc)

era5_variables = ["t2m", "u10", "v10", "ssrd", "tp"]
era5_data = get_era5_data(
    variables=era5_variables, country_code="AT", start=start, end=end
)
# actual_load_ts = get_load_data(start, end)

In [22]:
actual_load_ts

LoadTimeseries(data=                  actual_load_mw
period                          
2018-10-01 00:00          5256.0
2018-10-01 00:15          5204.0
2018-10-01 00:30          5178.0
2018-10-01 00:45          5151.0
2018-10-01 01:00          5187.0
...                          ...
2025-10-09 23:00          5713.0
2025-10-09 23:15          5642.0
2025-10-09 23:30          5584.0
2025-10-09 23:45          5516.0
2025-10-10 00:00          5548.0

[246337 rows x 1 columns], bidding_zone='BZN|AT')

In [74]:
era5_data["t2m"].sort_values(by="valid_time")

,value
valid_time,
2018-10-01 00:00:00+00:00,279.426453
2018-10-01 01:00:00+00:00,279.108917
2018-10-01 02:00:00+00:00,278.721954
2018-10-01 03:00:00+00:00,278.420013
2018-10-01 04:00:00+00:00,278.176880
...,...
2025-10-09 20:00:00+00:00,282.149750
2025-10-09 21:00:00+00:00,281.759949
2025-10-09 22:00:00+00:00,281.368591


In [72]:
era5_data["ssrd"].head(10)

,value
valid_time,
2018-10-01 00:00,0.000000e+00
2018-10-01 01:00,0.000000e+00
2018-10-01 02:00,0.000000e+00
2018-10-01 03:00,0.000000e+00
2018-10-01 04:00,2.055467e+01
2018-10-01 05:00,1.184000e+05
2018-10-01 06:00,4.094569e+05
2018-10-01 07:00,7.583427e+05
2018-10-01 08:00,1.041628e+06
